In [9]:
import requests
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Requests
- http://docs.python-requests.org/en/master/

##### 네이버 주식 데이터 가져오기
- api 사용 : json 파싱을 한다.
- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만들기
- http://m.stock.naver.com

In [10]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)

def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목", "시세", "전일비", "등락율", "시가총액", "거래량"])
    for company in companys:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"],
        }
    return df

In [11]:
url = make_url(100,1)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
95,포스코대우,22950,-1050,-4.37,28315,906590
96,동서,28150,-250,-0.88,28066,119864
97,만도,286000,9500,3.44,26859,109985
98,대림산업,76900,-1700,-2.16,26761,392396
99,메리츠화재,24250,-450,-1.82,26757,149029


##### Dark Sky API
- 날씨 정보를 알려주는 api
- https://darksky.net/dev
- pip3 install python-forecastio

In [24]:
import forecastio
FORECAST_TOKEN = "b41a8726e6a5f4e91e624f70eb66af47"

In [35]:
def forecast(lat,lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["hourly"]["summary"], json_info["timezone"]

In [36]:
lat = 37.5124413
lng = 126.9540519
forecast(lat,lng)

('Clear throughout the day.', 'Asia/Seoul')

In [37]:
def forecast(lat,lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHour = forecast.hourly()
    return byHour.summary

In [38]:
lat = 37.5124413
lng = 126.9540519
forecast(lat,lng)

'Clear throughout the day.'

### BeautifulSoup
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- pip3 install bs4

##### 네이버 검색어 순위
- bs4 사용 : html element를 selector를 사용한다.
- 네이버 검색어 순위를 가져와 데이터 프레임으로 만들기
- http://naver.com

In [29]:
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://naver.com")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".ah_roll .ah_l .ah_item")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    return df

In [30]:
naver_df = naver_top20()
naver_df

,rank,keyword
0,1,윤영주
1,2,이열음
2,3,몰디브
3,4,묵호항
4,5,국가장학금 소득분위 금액
5,6,청와대 국민청원
6,7,낸시랭
7,8,선미
8,9,오세훈
9,10,유웨이어플라이


##### 다음 검색어 순위

In [31]:
def daum_top10():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://daum.net")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#mArticle ol.list_hotissue.issue_row.list_mini > li")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text.replace("위",""),
            "keyword":keyword.select_one(".link_issue").text,
        }
    return df

In [32]:
daum_df = daum_top10()
daum_df

,rank,keyword
0,1,윤영주
1,2,묵호항
2,3,이상은
3,4,몰디브
4,5,이열음
5,6,최영미 시인
6,7,엠마 스톤
7,8,국가장학금 소득분위
8,9,정형식
9,10,최남수


##### 중복된 키워드 찾아서 출력하기

In [33]:
result = [keyword for keyword in daum_df["keyword"] if naver_df["keyword"].str.contains(keyword).any() ]
result

['윤영주', '묵호항', '몰디브', '이열음', '국가장학금 소득분위']

### file download

In [34]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)
    download_path = "./data/" + title
    size = 0
    with open(download_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                size += 1024
                f.write(chunk)
    return size

In [23]:
title = "iPhone X is Here — Apple.mp4"
download_link = "http://bit.ly/2FLpRF9"
size = download(title, download_link)
print("download done : {} Mbyte".format(round(size/1024/1024,2)))

FileNotFoundError: [Errno 2] No such file or directory: './data/iPhone X is Here — Apple.mp4'